In [ ]:
import tensorflow as tf
import numpy as np

# Define the 3D-GAN model architecture

def generator_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(4*4*4*256, use_bias=False, input_shape=(200,)))
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())

    model.add(tf.keras.layers.Reshape((4, 4, 4, 256)))
    assert model.output_shape == (None, 4, 4, 4, 256) 

    model.add(tf.keras.layers.Conv3DTranspose(128, (4, 4, 4), strides=(2, 2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 8, 8, 8, 128)
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())

    model.add(tf.keras.layers.Conv3DTranspose(64, (4, 4, 4), strides=(2, 2, 2), padding='same', use_bias=False))
    assert model.output_shape == (None, 16, 16, 16, 64)
    model.add(tf.keras.layers.BatchNormalization())
    model.add(tf.keras.layers.LeakyReLU())

    model.add(tf.keras.layers.Conv3DTranspose(1, (4, 4, 4), strides=(2, 2, 2), padding='same', use_bias=False, activation='sigmoid'))
    assert model.output_shape == (None, 32, 32, 32, 1)

    return model

def discriminator_model():
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Conv3D(64, (4, 4, 4), strides=(2, 2, 2), padding='same', input_shape=[32, 32, 32, 1]))
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Dropout(0.3))

    model.add(tf.keras.layers.Conv3D(128, (4, 4, 4), strides=(2, 2, 2), padding='same'))
    model.add(tf.keras.layers.LeakyReLU())
    model.add(tf.keras.layers.Dropout(0.3))

    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(1))

    return model

# Define the loss functions for the generator and discriminator
cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=True)

def discriminator_loss(real_output, fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    total_loss = real_loss + fake_loss
    return total_loss

def generator_loss(fake_output):
    return cross_entropy(tf.ones_like(fake_output), fake_output)

# Define the optimizers for the generator and discriminator
generator_optimizer = tf.keras.optimizers.Adam(0.0025, beta_1=0.5)
discriminator_optimizer = tf.keras.optimizers.Adam(1e-5, beta_1=0.5)

# Define the training loop for the 3D-GAN model
@tf.function
def train_step(generator, discriminator, images):
  noise = tf.random.normal([len(images), 200])
  with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
    generated_images = generator(noise, training=True)

    real_output = discriminator(images, training=True)
    fake_output = discriminator(generated_images, training=True)
    gen_loss = generator_loss(fake_output)
    disc_loss = discriminator_loss(real_output, fake_output)
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))

# Define the function to generate new 3D objects
def generate_objects(generator, num_objects=1):
  noise = tf.random.normal([num_objects, 200])
  generated_images = generator(noise, training=False)
  return generated_images.numpy()

# Define the function to test the 3D-GAN model
def test_3d_gan(generator, discriminator, dataset):
  num_objects = 10
  generated_images = generate_objects(generator, num_objects=num_objects)
  
  # Evaluate the discriminator on real and fake objects
  real_output = discriminator(dataset[:num_objects], training=False)
  fake_output = discriminator(generated_images, training=False)
  
  # Compute the accuracy of the discriminator
  real_accuracy = tf.reduce_mean(tf.cast(tf.greater(real_output, 0.5), tf.float32))
  fake_accuracy = tf.reduce_mean(tf.cast(tf.less(fake_output, 0.5), tf.float32))
  accuracy = (real_accuracy + fake_accuracy) / 2

  return accuracy.numpy()

# Load the dataset
IKEA_imgs_shapes = np.load('IKEA_imgs_shapes.npy')

# Initialize the generator and discriminator models
generator = generator_model()
discriminator = discriminator_model()

# Train the 3D-GAN model
num_epochs = 50
for epoch in range(num_epochs):
  for batch in IKEA_imgs_shapes:
    train_step(generator, discriminator, batch)
    
    
# Evaluate the performance of the 3D-GAN model
accuracy = test_3d_gan(generator, discriminator, IKEA_imgs_shapes)
print('Epoch {}, Discriminator Accuracy: {}'.format(epoch+1, accuracy))


# Generate new 3D objects with the trained generator
num_objects_to_generate = 10
generated_objects = generate_objects(generator, num_objects=num_objects_to_generate)

# Save the generated objects to file
np.save('generated_objects.npy', generated_objects)